In [27]:
# Generative methods vs Discriminative methods

In [28]:
import numpy as np
import pandas as pd
dataset = pd.read_csv('breast-cancer-wisconsin.data', header=None)

In [29]:
# data pre-processing
# take a look of the dataset, we can find the scalar of the first feature is so huge,so its better to do data pre-processing
print(dataset.dtypes)
# we find there are two col values are objects. we need to transform them into numbers
# after checking the dataset, we can find there are some missing data in col7

def find_missingdatarow_number(dataset):
    n = 0
    for i in range(data_num):
        if dataset.loc[i][6] == '?':
            n +=1 
    return n
data_num = dataset.shape[0]
missing_num = find_missingdatarow_number(dataset)
print("missing_num =",missing_num)
print("missing_rate =",missing_num/data_num)


0      int64
1      int64
2      int64
3      int64
4      int64
5      int64
6     object
7      int64
8      int64
9      int64
10     int64
dtype: object
missing_num = 16
missing_rate = 0.022889842632331903


In [30]:
# the missing rows rate is very low. so we can just delete the rows with missing values
def delete_missingdatarow_number(dataset):
    for i in range(data_num):
        if dataset.loc[i][6] == '?':
            dataset = dataset.drop(index = [i])
    return dataset
dt = delete_missingdatarow_number(dataset)
dt = dt.reset_index(drop = True)
print(dt)

          0   1   2   3   4   5   6   7   8   9   10
0    1000025   5   1   1   1   2   1   3   1   1   2
1    1002945   5   4   4   5   7  10   3   2   1   2
2    1015425   3   1   1   1   2   2   3   1   1   2
3    1016277   6   8   8   1   3   4   3   7   1   2
4    1017023   4   1   1   3   2   1   3   1   1   2
..       ...  ..  ..  ..  ..  ..  ..  ..  ..  ..  ..
678   776715   3   1   1   1   3   2   1   1   1   2
679   841769   2   1   1   1   2   1   1   1   1   2
680   888820   5  10  10   3   7   3   8  10   2   4
681   897471   4   8   6   4   3   4  10   6   1   4
682   897471   4   8   8   5   4   5  10   4   1   4

[683 rows x 11 columns]


In [31]:
X_dt = dt[[0,1,2,3,4,5,6,7,8,9]]
X_dt[6] = X_dt[6].astype(int)
print(X_dt.dtypes)
Y_dt = dt[10]
Y_dt = Y_dt.astype(int)


# Standrdlize
from sklearn.preprocessing import StandardScaler 
scaler = StandardScaler()
X_dt = scaler.fit_transform(X_dt)
from sklearn.model_selection import train_test_split
train_X,test_X,train_y,test_y = train_test_split(X_dt,Y_dt, test_size=0.2)


0    int64
1    int64
2    int64
3    int64
4    int64
5    int64
6    int64
7    int64
8    int64
9    int64
dtype: object


<ipython-input-31-fa757c8bd823>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_dt[6] = X_dt[6].astype(int)


In [32]:
print("train_X.shape:", train_X.shape)
print("test_X.shape:", test_X.shape)
print("train_y.shape:", train_y.shape)
print("test_y.shape:", test_y.shape)

train_X.shape: (546, 10)
test_X.shape: (137, 10)
train_y.shape: (546,)
test_y.shape: (137,)


In [33]:
# Pre-proccessing Finished

In [34]:
# add one col as bias term
ones_train = np.ones((train_X.shape[0], 1))
new_train_X = np.hstack((train_X,ones_train))
ones_test = np.ones((test_X.shape[0], 1))
new_test_X = np.hstack((test_X,ones_test))

x_tra = pd.DataFrame(new_train_X)
x_tes = pd.DataFrame(new_test_X)
y_tra = train_y.reset_index(drop = True)
y_tes = test_y.reset_index(drop = True)


In [35]:
# K-fold
from sklearn.model_selection import KFold
kf = KFold(n_splits=5,shuffle=True)

def log_likelihood(x, y, w):
    xw = np.dot(x, w)
    likelihood =  np.sum(y*xw - np.log(1 + np.exp(xw)))
    return likelihood

def sigmoid(a):
    return 1 / (1 + np.exp(-a))

def SGD(tra_x,tra_y, epoch_n, lr):     
    w = np.random.rand(tra_x.shape[1])  # 11 x 1
    train_sample_num = tra_x.shape[0]
    feature_num = tra_x.shape[1] 
    objval = 0
    for step in range(epoch_n):
        rand_indices = np.random.permutation(train_sample_num)
        tra_x_rand = tra_x.values[rand_indices, :]
        tra_y_rand = tra_y.values[rand_indices]
        for i in range(train_sample_num):
            xi = tra_x_rand[i,:]
            yi = tra_y_rand[i]
            predict = sigmoid(np.dot(xi, w))                
            y_minus_predict = yi - predict
            gradient = np.dot(xi.T, y_minus_predict)
            w += lr * gradient
            ll = log_likelihood(xi, yi, w)   
            objval += ll 
        objval = objval/train_sample_num
    print('log_likelihood_SGD after 50 epoch is', str(objval))
    return w

def Mini_batch_GD(tra_x,tra_y, epoch_n, lr,batch_size):
    w = np.random.rand(tra_x.shape[1])  # 11 x 1
    train_sample_num = tra_x.shape[0]
    feature_num = tra_x.shape[1] 
    objval = 0
    for step in range(epoch_n):
        rand_indices = np.random.permutation(train_sample_num)
        tra_x_rand = tra_x.values[rand_indices, :]
        tra_y_rand = tra_y.values[rand_indices]
        for i in range(0,train_sample_num,batch_size):
            xi = tra_x_rand[i:i+batch_size,:]
            yi = tra_y_rand[i:i+batch_size]
            predict = sigmoid(np.dot(xi, w))                
            y_minus_predict = yi - predict
            gradient = np.dot(xi.T, y_minus_predict)
            w += lr * gradient
            ll = log_likelihood(xi, yi, w)   
            objval += ll 
        objval = objval/train_sample_num
    print('log_likelihood_Minibatch_GD after 50 epoch is', str(objval))
    return w          

In [36]:
epoch_n = 50
lr = 0.05
w_folds_total_sgd = 0
w_folds_total_mini = 0
total_ll_sgd = 0
total_ll_mini = 0

# cross-validation
for train_index, val_index in kf.split(x_tra):
        tra_x,val_x = x_tra.loc[train_index],x_tra.loc[val_index]
        tra_y,val_y = y_tra[train_index],y_tra[val_index] 
        tra_y = (tra_y)/2 - 1
        val_y = (val_y)/2 - 1
#       perform SGD and get the w
        w_sgd = SGD(tra_x,tra_y, epoch_n, lr)
#       perform minibatch_GD and get the w
        w_minibatchgd = Mini_batch_GD(tra_x,tra_y, epoch_n,lr,batch_size = 20)    
        likelihood_on_valdata_sgd = log_likelihood(val_x,val_y,w_sgd)
        likelihood_on_valdata_mini = log_likelihood(val_x,val_y,w_minibatchgd)
        total_ll_sgd += likelihood_on_valdata_sgd
        total_ll_mini += likelihood_on_valdata_mini     
        w_folds_total_sgd += w_sgd
        w_folds_total_mini += w_minibatchgd
# avrage the w and loss function result
w_star_sgd = w_folds_total_sgd/5
w_star_mini = w_folds_total_mini/5
total_ll_sgd = total_ll_sgd/5
total_ll_mini = total_ll_mini/5

print("log_likelihood with SGD is:",total_ll_sgd)
print("log_likelihood with Minibatch_gd is:",total_ll_mini)

log_likelihood_SGD after 50 epoch is -0.06934480134371307
log_likelihood_Minibatch_GD after 50 epoch is -0.07014106328575777
log_likelihood_SGD after 50 epoch is -0.0699295719211398
log_likelihood_Minibatch_GD after 50 epoch is -0.0702587072600456
log_likelihood_SGD after 50 epoch is -0.04800054339285878
log_likelihood_Minibatch_GD after 50 epoch is -0.0487559925426709
log_likelihood_SGD after 50 epoch is -0.07533928636551158
log_likelihood_Minibatch_GD after 50 epoch is -0.07468737256544962
log_likelihood_SGD after 50 epoch is -0.06748093514868099
log_likelihood_Minibatch_GD after 50 epoch is -0.06789398763255682
log_likelihood with SGD is: -10.888875638389559
log_likelihood with Minibatch_gd is: -10.88935396537002


In [37]:
# from the results, we can see SGD and Minibatch_Gd works perfect on our model with hyperparameter lr= 0.05,epoch = 50

In [38]:
# recall, precision, and accuracy on (1) class prediction on the test data
# In this model, SGD and Mini Batch GD performs same well, so i'm gonna use SGD to do the test .
y_predicted = sigmoid(x_tes.dot(w_star_sgd))
test_len = y_predicted.shape[0]
print(test_len)
for i in range(test_len):
    if y_predicted[i] < 0.5:
        y_predicted[i] = 0
    else:
        y_predicted[i] = 1
print(y_predicted)
            




137
0      0.0
1      0.0
2      0.0
3      0.0
4      0.0
      ... 
132    0.0
133    0.0
134    1.0
135    1.0
136    0.0
Length: 137, dtype: float64


In [39]:
# cuz in original dataset, the label is either 2 or 4 but we need o and 1.
y_tes = y_tes/2 - 1
print(y_tes)

0      0.0
1      0.0
2      0.0
3      0.0
4      0.0
      ... 
132    0.0
133    0.0
134    1.0
135    1.0
136    0.0
Name: 10, Length: 137, dtype: float64


In [43]:
right_class1 = 0
right_class2 = 0
wrong_class1 = 0
wrong_class2 = 0
for i in range(test_len):
    if y_predicted[i] == y_tes[i] and y_predicted[i] == 0 :
        right_class1+=1
    if y_predicted[i] == y_tes[i] and y_predicted[i] == 1 :
        right_class2+=1
    if y_predicted[i] == 0 and y_tes[i] == 1 :
        wrong_class1+=1
    if y_predicted[i] == 1 and y_tes[i] == 0 :
        wrong_class2+=1
print("right_class1 :",right_class1)
print("right_class2 :",right_class2)
print("wrong_class1 :",wrong_class1)
print("wrong_class2 :",wrong_class2)

right_class1 : 81
right_class2 : 49
wrong_class1 : 5
wrong_class2 : 2


In [45]:
# For malignant class
precision = right_class2 / (right_class2+wrong_class2)
recall = right_class2 / (right_class2+wrong_class1)
accuracy = (right_class1+right_class2) / test_len
print("precision for malignant on testset :",precision)
print("recall for malignant on testset :",recall)
print("accuracy for the model on testset :",accuracy)

precision for malignant on testset : 0.9607843137254902
recall for malignant on testset : 0.9074074074074074
accuracy for the model on testset : 0.948905109489051


In [ ]:
# Our Model is not bad !